In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download zh_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install jieba

In [ ]:
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=8612e120846d8212131c4a78d7f9604e932b8790ea34e58bd5846109a198b156
  Stored in directory: /root/.cache/pip/wheels/43/f3/70/8990fc249efeb396007766676706f71dd3d1ca3c023ce522ce
Successfully built snownlp


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
import spacy
import re
import string
import jieba
from snownlp import SnowNLP

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
# red the csv datasets
train_df = pd.read_csv('train_zh_dataset.csv')
test_df = pd.read_csv('test_zh_dataset.csv')

In [ ]:
class TweetDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['comment_text']
        label = self.data.iloc[idx]['label']
        return text, label

In [ ]:
train_dataset = TweetDataset(train_df)
test_dataset = TweetDataset(test_df)

In [ ]:
# show the frist five samples in this dataset
train_dataset[0:5]

(0         其实我觉得也不能太偏激了吧。我们男性不说不代表我们不知道对错，只是不喜欢去评论这些事情。
 1                       不完全统计，十三个伏地魔相关博主被炸号，其中包括一位维权素人
 2    只是从图二里表达出来的是那些发达国家，我也没有不尊重其他国家，只是觉得一味地崇洋媚外，甚至说...
 3             其他的不说 对待舆论的态度真的圈粉 不卑不亢 掷地有声:green_heart:
 4    男人也吃男人，也有男吃女女吃男，怎么就毫无存在感了？单独拿出来说女吃女，仿佛是为了证明女性的...
 Name: comment_text, dtype: object,
 0    0
 1    0
 2    0
 3    0
 4    1
 Name: label, dtype: int64)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
nlp = spacy.load("zh_core_web_sm")

In [ ]:
# Chinese women-hatred lexicon - chinese
with open('SexHateLex.txt', 'r', encoding='utf-8') as file:
    SexHateLexicon = [line.strip() for line in file]

In [ ]:
punctuation_pattern = r"[，。？！：；、“”‘’（）《》【】——……—·,.\?!:;'\"()<>]"

In [ ]:
def handcrafted_features(texts):
    features = []
    for text in texts:
        doc = nlp(text)

        # clauses per sentence
        sentence_count = len(list(doc.sents))
        total_clauses = 0
        for sent in doc.sents:
            clause_count = sent.text.count('，') + 1
            total_clauses += clause_count
        clause_per_sentence = total_clauses / sentence_count if sentence_count > 0 else 0

        # count of imperative sentences
        # define the list of words that suuggest imparatives
        imperative_words = {"请", "务必", "不要", "别", "一定", "千万"}
        mood_particles = {"吧", "啦", "呀"}

        imperative_count = sum(
            1 for sent in doc.sents
            if (
                any(word.text in imperative_words for word in sent) or
                (len(sent) > 0 and sent[-1].text in mood_particles)
        )
    )

        # Count of passive voice usage (using words like "被"bei which often indicates passive)
        passive_count = sum(1 for token in doc if token.text == "被")

        # Gendered pronoun ratio for written Chinese
        pronouns = [token.text for token in doc if token.pos_ == "PRON"]
        women_gendered_pronouns = {'她','她们'}  # Female pronoun in Chinese
        gendered_count = sum(1 for pronoun in pronouns if pronoun in women_gendered_pronouns)
        total_pronouns = len(pronouns)
        gendered_pronoun_ratio = gendered_count / total_pronouns if total_pronouns > 0 else 0

        # Count of negations (e.g., "不", "没")
        neg_count = sum(1 for token in doc if token.text in {"不", "没", "别", "否"})

        # tokenize the Chinese weibo using jieba tokenlization package
        tokens = list(jieba.cut(text))

        # work number in each tweet
        token_num_per_tweet = len(tokens)

        # avg word length in each tweet
        char_num_per_tweet = sum(len(token) for token in tokens)
        avg_char_num_per_token = char_num_per_tweet / token_num_per_tweet if token_num_per_tweet != 0 else 0

        # number of sentences in each tweet
        sentences = re.split(r'[。！？]', text)
        sentences = [s for s in sentences if s.strip()]
        sentence_num = len(sentences)

        # # avg character number per sentence in a tweet
        # avg_char_num_per_sentence = char_num_per_tweet / sentence_num if sentence_num != 0 else 0

        # 4. number of hastags
        hashtag_num = len(re.findall(r'#(?!URL\b)\w+', text))

        # 5. number of mentions
        mention_num = text.count('@username')

        # 6. number of links
        link_num = text.count('#URL')
        # sentiment analysis
        s = SnowNLP(text)
        # this score ranges from 0 to 1
        sentiment_compound = s.sentiments

        # sexism word frequency statistics
        sexwords_count = len([token for token in tokens if token in SexHateLexicon])
        # 9. ratio of sexist word in a tweet
        sexwords_ratio = sexwords_count / token_num_per_tweet if token_num_per_tweet > 0 else 0

        # 10. number of all punctuations of each tweet
        punctuation_count = len(re.findall(punctuation_pattern, text))

        # 11. ratio of punctuations in relation to the number of words
        punctuation_ratio = punctuation_count / token_num_per_tweet if token_num_per_tweet > 0 else 0

        # 12. number of exclamation marks
        exclamation_count = text.count('！')

        # 13. ratio of exclamation marks
        exclamation_ratio = exclamation_count / token_num_per_tweet if token_num_per_tweet > 0 else 0

        # 14. number of question marks
        question_count = text.count('？')

        # 15. ratio of question marks
        question_ratio = question_count / token_num_per_tweet if token_num_per_tweet > 0 else 0

        # count of emojis in each tweet
        emoji_count = len(re.findall(r':[^:]+?:', text))

        # 17. emoji ratio
        emoji_ratio = emoji_count / token_num_per_tweet if token_num_per_tweet > 0 else 0

        # append features for each weibo as a list
        features.append([
            clause_per_sentence,
            imperative_count,
            passive_count,
            gendered_pronoun_ratio,
            neg_count,
            token_num_per_tweet,
            avg_char_num_per_token,
            sentence_num,
            hashtag_num,
            mention_num,
            link_num,
            sentiment_compound,
            sexwords_count,
            sexwords_ratio,
            punctuation_count,
            punctuation_ratio,
            exclamation_count,
            exclamation_ratio,
            question_count,
            question_ratio,
            emoji_count,
            emoji_ratio
        ])
    # convert to tensor
    return torch.tensor(features, dtype=torch.float32)

In [ ]:
# build the integrated model class
class IntegratedSexistDetector(nn.Module):
    def __init__(self, padding='max_length', num_classes=1, handcrafted_feature_dim=22):
        super(IntegratedSexistDetector, self).__init__()
        self.berttokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.padding = padding

        combined_feature_dim = self.bert.config.hidden_size + self.bert.config.hidden_size + handcrafted_feature_dim
        self.cls = nn.Sequential(
            nn.Linear(combined_feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(256, num_classes),
            nn.Sigmoid()
        )
        # set the bert parameters non-trainable
        for param in self.bert.parameters():
            param.requires_grad = False

    def tokenize(self, texts):
        encoding = self.berttokenizer(
            texts,
            add_special_tokens=True,
            padding=self.padding,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        return input_ids, attention_mask

    def forward(self, texts):
        input_ids, attention_mask = self.tokenize(texts)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.pooler_output
        # select the real tokens in a tweet, excluding special tokens
        hidden_state = outputs.last_hidden_state[:,1:-1,:]
        mean_hidden_state = self.pooling(hidden_state.permute(0, 2, 1)).permute(0, 2, 1).squeeze(1)
        handcrafted_feats = handcrafted_features(texts).to(device)
        # fuse these the pooler output, mean hidden state and the hand-crafted features together by direct concatenation
        combined_features = torch.cat([cls_token, mean_hidden_state, handcrafted_feats], dim=1)
        logits = self.cls(combined_features)
        return logits

In [ ]:
# train function
def train(model, train_loader, test_loader, optimizer,
          scheduler,
          epochs, device, criterion=nn.BCELoss()):
    best_acc = 0
    model.train()

    for epoch in range(epochs):
        total_loss = 0

        # training loop
        for (texts, labels) in tqdm(train_loader):
            labels = labels.to(torch.float32).to(device)
            optimizer.zero_grad()
            logits = model(texts)
            logits = logits.squeeze(1)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        # evaluate the model on the validation set after each epoch
        acc, f1 = evaluate(model, test_loader, device)
        print(f"Test Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")

        # if current acc is greater than previous best acc, save a new best model
        if acc > best_acc:
            best_acc = acc
            print(f"New best model found with accuracy: {best_acc:.4f}, saving the model...")
            torch.save(model, "best_model.pth")

        # apply scheduler to adjust the learning rate
        scheduler.step()

    print("Training complete!")

In [ ]:
# evaluate model
def evaluate(model, dataloader, device, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for (texts, labels) in tqdm(dataloader):
            labels = labels.to(device)
            logits = model(texts)
            # ensure logits is squeezed to have a single value per example
            logits = logits.squeeze(1)
            preds = (logits > threshold).int()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, f1

In [ ]:
model = IntegratedSexistDetector()
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

IntegratedSexistDetector(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
epochs = 50

In [ ]:
train(model, train_loader, test_loader, optimizer, scheduler, epochs, device)

  0%|          | 0/449 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.736 seconds.
DEBUG:jieba:Loading model cost 0.736 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
100%|██████████| 449/449 [03:37<00:00,  2.07it/s]


Epoch 1/50, Loss: 0.5279


100%|██████████| 113/113 [00:53<00:00,  2.10it/s]


Accuracy: 0.7832
F1 Score: 0.6845
Test Accuracy: 0.7832, F1 Score: 0.6845
New best model found with accuracy: 0.7832, saving the model...


100%|██████████| 449/449 [03:34<00:00,  2.09it/s]


...

Accuracy: 0.7965
F1 Score: 0.7025
Test Accuracy: 0.7965, F1 Score: 0.7025


100%|██████████| 449/449 [03:33<00:00,  2.10it/s]


Epoch 50/50, Loss: 0.2741


100%|██████████| 113/113 [00:53<00:00,  2.10it/s]

Accuracy: 0.7960
F1 Score: 0.7034
Test Accuracy: 0.7960, F1 Score: 0.7034
Training complete!


In [ ]:
# load the best model
best_integrated_model = torch.load('best_model.pth').to(device)

<ipython-input-27-145e4e1744e9>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_integrated_model = torch.load('best_model.pth').to(device)


In [ ]:
evaluate(best_integrated_model, test_loader, device)

100%|██████████| 113/113 [00:53<00:00,  2.10it/s]

Accuracy: 0.7977
F1 Score: 0.7103


(0.7976588628762542, 0.7102952913008779)